In [1]:
# Utils
import torch
import numpy as np

# ML models
from openxai.LoadModel import LoadModel

# Data loaders
from openxai.dataloader import return_loaders

# Explanation models
from openxai.Explainer import Explainer

# Evaluation methods
from openxai.evaluator_final import Evaluator

# Perturbation Methods
from openxai.explainers.catalog.perturbation_methods import NormalPerturbation
from openxai.explainers.catalog.perturbation_methods import NewDiscrete_NormalPerturbation

In [2]:
# Choose the model and the data set you wish to generate explanations for
data_loader_batch_size = 32
data_name = 'adult' 
model_name = 'lr'

### (0) Explanation method hyperparameters

In [3]:
# Hyperparameters for Lime
lime_mode = 'tabular'
lime_sample_around_instance = True
lime_kernel_width = 0.75
lime_n_samples = 1000
lime_discretize_continuous = False
lime_standard_deviation = float(np.sqrt(0.03))

### (1) Data Loaders

In [4]:
# Get training and test loaders
loader_train, loader_test = return_loaders(data_name=data_name,
                                           download=True,
                                           batch_size=data_loader_batch_size)
data_iter = iter(loader_test)
inputs, labels = data_iter.next()

In [5]:
# get full training data set
data_all = torch.FloatTensor(loader_train.dataset.data)

### (2) Load a pretrained ML model

In [6]:
# Load pretrained ml model
model = LoadModel(data_name=data_name,
                  ml_model=model_name,
                  pretrained=True)

### (3) Choose an explanation method

#### I: Explanation method with particular hyperparameters (LIME)

In [7]:
# You can supply your own set of hyperparameters like so:
param_dict_lime = dict()
param_dict_lime['dataset_tensor'] = data_all
param_dict_lime['std'] = lime_standard_deviation
param_dict_lime['mode'] = lime_mode
param_dict_lime['sample_around_instance'] = lime_sample_around_instance
param_dict_lime['kernel_width'] = lime_kernel_width
param_dict_lime['n_samples'] = lime_n_samples
param_dict_lime['discretize_continuous'] = lime_discretize_continuous
lime = Explainer(method='lime',
                 model=model,
                 dataset_tensor=data_all,
                 param_dict_lime=param_dict_lime)

In [8]:
lime_custom = lime.get_explanation(inputs, 
                                   label=labels)

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 87.19it/s]


In [9]:
lime_custom[0,:]

tensor([ 1.6443e-01,  1.2201e-02,  3.7314e-01,  2.0040e+00,  2.1571e-01,
         1.8303e-01,  2.5536e-02,  2.3337e-02, -2.4086e-01, -4.8734e-02,
         4.5277e-02,  1.9673e-02,  1.9051e-03])

#### II: Explanation method with default hyperparameters (LIME)

In [10]:
# You can also use the default hyperparameters likes so:
lime = Explainer(method='lime',
                 model=model,
                 dataset_tensor=data_all,
                 param_dict_lime=None)
lime_default = lime.get_explanation(inputs.float(), 
                                    label=labels)
lime_default[0,:]

100%|█████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 104.97it/s]


tensor([ 1.1277e-01,  1.3031e-02,  3.0284e-01,  1.6160e+00,  1.8914e-01,
         1.5675e-01, -9.7942e-04, -6.0445e-03, -1.7885e-01, -4.7259e-02,
         1.8248e-02, -3.6790e-03, -8.5986e-03])

#### III: Explanation method with default hyperparameters (IG)

In [11]:
# To use a different explanation method change the method name like so
ig = Explainer(method='ig',
               model=model,
               dataset_tensor=data_all,
               param_dict_lime=None)
ig_default_exp = ig.get_explanation(inputs.float(), 
                                    label=labels)
ig_default_exp[0,:]

tensor([-2.8762e-01, -3.7618e-02, -7.2914e-01, -3.9698e+00, -4.4084e-01,
        -3.7505e-01, -4.5641e-02,  7.7510e-04,  4.5545e-01,  9.8296e-02,
        -4.6579e-02, -1.3360e-02, -8.3665e-03], dtype=torch.float64)

### (4) Choose an evaluation metric

In [12]:
def generate_mask(explanation, top_k):
    mask_indices = torch.topk(explanation, top_k).indices
    mask = torch.zeros(explanation.shape) > 10
    for i in mask_indices:
        mask[i] = True
    return mask

In [13]:
# Perturbation class parameters
perturbation_mean = 0.0
perturbation_std = 0.05
perturbation_flip_percentage = 0.03
feature_types = ['c', 'c', 'c', 'c', 'c', 'c', 'd', 'd', 'd', 'd', 'd', 'd', 'd']

perturbation = NormalPerturbation("tabular",
                                  mean=perturbation_mean,
                                  std_dev=perturbation_std,
                                  flip_percentage=perturbation_flip_percentage)

In [14]:
input_dict = dict()
index = 0

# inputs and models
input_dict['x'] = inputs[index].reshape(-1)
input_dict['input_data'] = inputs
input_dict['explainer'] = ig
input_dict['explanation_x'] = ig_default_exp[index,:].flatten()
input_dict['model'] = model

# perturbation method used for the stability metric
input_dict['perturbation'] = perturbation
input_dict['perturb_method'] = perturbation
input_dict['perturb_max_distance'] = 0.4
input_dict['feature_metadata'] = feature_types
input_dict['p_norm'] = 2
input_dict['eval_metric'] = None

# true label, predicted label, and masks
input_dict['top_k'] = 10
input_dict['y'] = labels[index].detach().item()
input_dict['y_pred'] = torch.max(model(inputs[index].unsqueeze(0).float()), 1).indices.detach().item()
input_dict['mask'] = generate_mask(input_dict['explanation_x'].reshape(-1), input_dict['top_k'])

# required for the representation stability measure
input_dict['L_map'] = model

In [15]:
evaluator = Evaluator(input_dict,
                      inputs=inputs,
                      labels=labels, 
                      model=model, 
                      explainer=ig)

In [16]:
# evaluate rank correlation
evaluator.evaluate(metric='PRA')

(array([1.]), 1.0)

In [17]:
# evaluate rank correlation
evaluator.evaluate(metric='RC')

(array([1.]), 0.9999999999999999)

In [18]:
# evaluate feature agreement
evaluator.evaluate(metric='FA')

(array([1.]), 1.0)

In [19]:
# evaluate rank agreement
evaluator.evaluate(metric='RA')

(array([1.]), 1.0)

In [20]:
# evaluate sign agreement
evaluator.evaluate(metric='SA')

(array([0.]), 0.0)

In [21]:
# evaluate signed rankcorrelation
evaluator.evaluate(metric='SRA')

(array([0.]), 0.0)

In [22]:
# evaluate prediction gap on umportant features
evaluator.evaluate(metric='PGU')

0.30328226

In [23]:
# evaluate prediction gap on important features
evaluator.evaluate(metric='PGI')

0.5746145

In [24]:
# evaluate relative input stability
evaluator.evaluate(metric='RIS')

42.29351355908528

In [25]:
# evaluate relative output stability
evaluator.evaluate(metric='ROS')

2244.6194486691425